In [1]:
!pip install simpy

In [14]:
import simpy
import random
import numpy as np

# Parâmetros do sistema
LOGGER = False
TAXA_CHEGADA = 0.5   # Taxa de chegada (λ)
TAXA_SERVICO = 1.0   # Taxa de serviço (µ)
NUM_SERVIDORES = 1   # Número de servidores
TEMPO_SIMULACAO = 100000
tempos_de_espera = []

# Função para chegada de clientes
def cliente(env, nome, servidor):
    chegada = env.now

    if LOGGER:
      print(f'{nome} chegou em {chegada:.2f}')

    # Cliente pede o servidor
    with servidor.request() as req:
        yield req  # Espera até ser atendido
        inicio_servico = env.now

        tempos_de_espera.append(inicio_servico - chegada) # Calcula o tempo de espera e guarda no array de tempos de espera

        if LOGGER:
          print(f'{nome} começou o atendimento em {inicio_servico:.2f}')

        # Tempo de atendimento
        tempo_atendimento = random.expovariate(TAXA_SERVICO)
        yield env.timeout(tempo_atendimento)

        fim_servico = env.now
        if LOGGER:
          print(f'{nome} terminou o atendimento em {fim_servico:.2f}')

# Função para gerar chegadas
def gerar_clientes(env, servidor):
    contador = 0
    while True:
        yield env.timeout(random.expovariate(TAXA_CHEGADA))  # Tempo entre chegadas
        contador += 1
        env.process(cliente(env, f'Cliente {contador}', servidor))

# Função principal
def simular():
    env = simpy.Environment()  # Ambiente de simulação
    servidor = simpy.Resource(env, capacity=NUM_SERVIDORES)  # Servidor (com 1 atendente)
    env.process(gerar_clientes(env, servidor))  # Inicia o processo de geração de clientes
    env.run(until=TEMPO_SIMULACAO)  # Executa a simulação por X unidades de tempo

    print(f'Tempo médio de espera (W): {np.mean(tempos_de_espera)}')
    print(f'Variância do tempo médio de espera var(W): {np.var(tempos_de_espera)}')


if __name__ == '__main__':
    simular()

Tempo médio de espera (W): 0.9858921568870023
Variância do tempo médio de espera var(W): 2.8085855968988254
